In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import StatPol as SP

# Analysis of the StatPol module

This nb describe the usage of the StaticalPolarizability module that allows us to compute the statical polarizability of a system and to perform convergence analysis w.r.t. the intensity of the static field and the size of the simulation domain

We assume that the GS analysis for a set of molecule has been performed. The structure of this working directory consists in a folder for each molecule. Inside each folder there is one logifle (or more if different study varying for instance xc functional or equilibrium position) that contains the result of the GS computation used as input of the present analysis

Scan the path folder and build the molecules_database by looking at the directories in the path

In [2]:
molecules_database = SP.get_molecule_database()
molecules_database

['H20', 'CO']

Define the dictionary that contains the polarizability data of the molecules

In [3]:
polTensor_data = {}

Choose a molecule to perform the analysis, move the path in the associated folder and get the relevan parameters of the gs analysis

In [4]:
molecule = molecules_database[1]
%cd $molecule
sys.path.insert(0,'../')
posinp_file = molecule+'_posinp.xyz'
gs = lf.Logfile('log-lda.yaml')

/home/marco/Data/RICERCA/LINEAR RESPONSE/LR-nb/STATICAL POLARIZABILITY/CO


## Computation of alpha for a specific choice of the input parameters

Build the input file and define the parameters common to all the runs.

The values of rmult and of the field intensity will be specified subsequently since a convergence study will be performed on this parameters.

As a first usage of the module we compute the statical polarizabiltiy tensor for a specific choice of the field intensity and the simulation domain

In [5]:
fieldInt = 1e-2

inp = I.Inputfile()
inp.set_hgrid(gs.log['dft']['hgrids'])
inp.set_xc('LDA')
inp.set_wavefunction_convergence(gnrm=1.0e-5)
inp.set_rmult(gs.log['dft']['rmult'])
inp

{'dft': {'gnrm_cv': 1e-05, 'hgrids': 0.37, 'ixc': 'LDA', 'rmult': [7.0, 9.0]}}

In [6]:
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True) #,verbose=False

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


In [7]:
reload(SP)
a = SP.build_alpha_dataset(intensity=fieldInt,input=inp,runner=code,posinp=posinp_file,ppf=SP.eval_alpha)
# display some info on a
write(a.global_options())
ind = 0
write(a.runs[ind])

{'run_dir': 'alpha', 'intensity': 0.01, 'posinp': 'CO_posinp.xyz', 'label': 'alpha_0.01'}
{'run_dir': 'alpha', 'input': {'dft': {'ixc': 'LDA', 'gnrm_cv': 1e-05, 'elecfield': [0.01, 0.0, 0.0], 'hgrids': 0.37, 'rmult': [7.0, 9.0]}}, 'intensity': 0.01, 'posinp': 'CO_posinp.xyz', 'label': 'alpha_0.01'}


In [8]:
alpha = a.run()
alpha

Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:x,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:y,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:y,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:z,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:z,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:-.yaml"
Run directory alpha
Executing co

matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
        [-1.279955e-05,  1.323614e+01, -3.389697e-05],
        [-1.125000e-04, -1.125000e-04,  1.856809e+01]])

The verbose option of SystemCalculator can be used to limit the display on terminal

In [9]:
code.update_global_options(verbose=False)
alpha = a.run()
alpha

matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
        [-1.279955e-05,  1.323614e+01, -3.389697e-05],
        [-1.125000e-04, -1.125000e-04,  1.856809e+01]])

Attempt for a general convergence function (or class???)

Comincio a definire una funzione che prende come input un array con i valori del parametro di convergenza, un array con i tre dataset associati, una label con il nome del parametro e le tolleranze. 
Che cosa ritorna? Forse conviene un dizionario con i parametri di input, i risultati del run del dataset per tutti i valori, il valore di convergenza e un boleano che dice se la convergenza è raggiunta o no

In [63]:
def seek_convergence(at=1e-3,rt=1e-2,**kwargs):
    """
    Perform a convergence procedure by using 3 values of a parameter.
    Return a dictionary with the input parameters, the results of all
    the computation performed, the value of the convergence parameter and a
    boolean that states if the convergence procedure succeeds or not
    
    Args:
        kwargs['label']     : the name of the convergence parameter
        kwargs['values']    : the array with the 3 ordered values of the convergence parameter
        kwargs['data']      : the array with the dataset buit with kwargs['values']
        at,rt               : absolute and relative tol of np.allclose
    """
    label = kwargs['label']
    values = kwargs['values']
    data = kwargs['data']
    results = {}
    for ind,v in enumerate(values[:2]):    
        print 'Run the dataset with', label, v
        results[v] = data[ind].run()
    
    out = {'label':label,'values':values}
    
    convergence = np.allclose(results[values[0]],results[values[1]],atol = at, rtol = rt)
    if convergence:
        write('Convergence achieved for', label ,values[0])
        results[values[2]] = None
        out['results'] = results
        out['converged'] = True
        out['converged_value'] = values[0]
    else:
        write('Convergence for', label,values[0],'failed')
        write('Set the value of the parameter to', values[1])
        write('')
        print 'Run the dataset with', label, values[2]
        results[values[2]]=data[2].run()
        out['results'] = results
        convergence = np.allclose(results[values[1]],results[values[2]],atol = at, rtol = rt)
        if convergence:
            write('Convergence achieved for', label ,values[1])
            out['converged'] = True
            out['converged_value'] = values[1]
        else:
            write('Convergence for', label,values[1],'failed')
            write('Return the value associated to',label,values[2],'. Perform further check!!!')
            out['converged'] = False
            out['converged_value'] = values[2]

    return out

In [66]:
field_int = [1e-2,5e-3,1e-3]
data = []
code.update_global_options(verbose=False)
for f in field_int:
    data.append(SP.build_alpha_dataset(intensity=f,input=inp,runner=code,posinp=posinp_file,ppf=SP.eval_alpha))
    
r = seek_convergence(rt=1e-2,label='field_int',values=field_int,data=data)
r

Run the dataset with field_int 0.01
Run the dataset with field_int 0.005
Convergence achieved for field_int 0.01


{'converged': True,
 'converged_value': 0.01,
 'label': 'field_int',
 'results': {0.001: None,
  0.005: matrix([[ 1.3211721e+01, -4.5186718e-04, -2.4358770e-05],
          [-4.5186719e-04,  1.3211721e+01, -2.4358770e-05],
          [ 1.0600000e-04,  1.0600000e-04,  1.8530906e+01]]),
  0.01: matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
          [-1.279955e-05,  1.323614e+01, -3.389697e-05],
          [-1.125000e-04, -1.125000e-04,  1.856809e+01]])},
 'values': [0.01, 0.005, 0.001]}

## Convergence analysis w.r.t the intensity of the field

Perform a comparison among the alpha computed with different field intensity of 1e-2 and 5e-3 and perform a further computation with a field equal to 1e-3 if the alphas differ more than a given tolerance. The comparison is performed with np.allclose functions that returns true if

abs(a-b) < atol + rtol*abs(b)

So for small elements that are deeply affected by numerical noise the maximum allowed discrepacy is atol while for the relevan ones the percentage difference is rtol. 

Define the function that performs the convergence of alpha w.r.t. the intensity of the field

In [15]:
def perform_field_convergence(at = 1e-3, rt = 1e-2,**kwargs):
    """
    Perform the corvergence procedure w.r.t. the intensity of the static field to extract the
    result of the polarizability tensor. 
    Return a dictionary with the polarizability tensor, the value of the field at which is
    calculated and a boolean variable that states if the convergence procedure succeds or not
    
    Args:
        kwargs['input']     : the input file
        kwargs['posinp']    : the posinp
        kwargs['ppf']       : the postprocessing function
        kwargs['runner']    : the instance of SystemCalculator
        at,rt               : absolute and relative tol of np.allclose
    """
    field_int = [1e-2,5e-3,1e-3]
    alphas = {}
    out = {}
    for f in field_int[:2]:
        a = build_alpha_dataset(intensity=f,input=kwargs['input'],runner=kwargs['runner'],posinp=kwargs['posinp'],ppf=kwargs['ppf'])
        alphas[f] = a.run()
    
    convergence = np.allclose(alphas[field_int[0]],alphas[field_int[1]],atol = at, rtol = rt)
    if convergence:
        write('')
        write('Convergence achieved for field intensity ',field_int[0])
        write('')
        out = {'alpha' : alphas[field_int[0]],'F' : field_int[0], 'f_convergence' : True} 
    else:
        write('')
        write('Convergence for field intensity ',field_int[0],' failed')
        write('Reduce the value of the field')
        write('')
        a = build_alpha_dataset(intensity=field_int[2],input=kwargs['input'],runner=kwargs['runner'],posinp=kwargs['posinp'],ppf=kwargs['ppf'])
        alphas[field_int[2]] = a.run()
        convergence = np.allclose(alphas[field_int[1]],alphas[field_int[2]],atol = at, rtol = rt)
        if convergence:
            write('')
            write('Convergence achieved for field intensity ',field_int[1])
            write('')
            out = {'alpha' : alphas[field_int[1]],'F' : field_int[1], 'f_convergence' : True} 
        else:
            write('')
            write('Convergence for field intensity ',field_int[1],' failed')
            write('Return the value of alpha associated to ',field_int[2],'. Perform further check!!!')
            write('')
            out = {'alpha' : alphas[field_int[2]],'F' : field_int[2], 'f_convergence' : False} 
    return out

In [16]:
inp.set_rmult(gs.log['dft']['rmult'])
out = perform_field_convergence(rt=1e-2,input=inp,runner=code,posinp=posinp_file,ppf=eval_alpha)
out


Convergence achieved for field intensity  0.01



{'F': 0.01, 'alpha': matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
         [-1.279955e-05,  1.323614e+01, -3.389697e-05],
         [-1.125000e-04, -1.125000e-04,  1.856809e+01]]), 'f_convergence': True}

The last step of this analysis is the convergence of alpha w.r.t. the size of the box

In [17]:
def build_rmult_list(gs):
    """
    Return a set of values of rmult. The set starts at the value of rmult of the gs
    calculation and contains 3 values to perform a convergence procedure analogous 
    to the one performed w.r.t. the field intensity
    """
    
    r0 = gs.log['dft']['rmult']
    rmult_set = []
    for incr in range(3):
        rmult_set.append([r0[0]+incr,r0[1]])
    return rmult_set

def perform_rmult_convergence(gs_log = gs,at = 1e-3, r_rtol = 1e-2,f_rtol = 1e-2,**kwargs):
    """
    Perform the convergence procedure w.r.t. the size of the simulation domain to extract the
    result of the polarizability tensor. 
    Return a dictionary with the value of rmult, the results of the field convergence procedure 
    and a boolean variable that states if the convergence procedure succedes or not
    
    Args:
        gs_log              : log of the gs calculation
        kwargs['input']     : the input file
        kwargs['posinp']    : the posinp
        kwargs['ppf']       : the postprocessing function
        kwargs['runner']    : the instance of SystemCalculator
        at                  : absolute tol of np.allclose
        f_rtol,r_rtol       : relative tol of np.allclose for f and r convergence
    """
    alphas = {}
    out = {}
    
    print 'r_rtol', r_rtol
    print 'f_rtol', f_rtol
    
    rmult = build_rmult_list(gs)
    for r in rmult[:2]:
        kwargs['input'].set_rmult(r)
        alphas[r[0]]=perform_field_convergence(at,rt=f_rtol,input=kwargs['input'],runner=kwargs['runner'],posinp=kwargs['posinp'],ppf=kwargs['ppf'])
    
    convergence = np.allclose(alphas[rmult[0][0]]['alpha'],alphas[rmult[1][0]]['alpha'],atol = at, rtol = r_rtol)
    if convergence:
        write('')
        write('Convergence achieved for rmult value : ',rmult[0][0])
        write('')
        out = {'rmult':rmult[0][0],'r_convergence':True,'results':alphas[rmult[0][0]]}
    else:
        write('')
        write('Convergence for rmult value ',rmult[0][0],' failed')
        write('Increase the size of the box')
        write('')
        kwargs['input'].set_rmult(rmult[2])
        alphas[rmult[2][0]]=perform_field_convergence(at,rt=f_rtol,input=kwargs['input'],runner=kwargs['runner'],posinp=kwargs['posinp'],ppf=kwargs['ppf'])
        convergence = np.allclose(alphas[rmult[1][0]]['alpha'],alphas[rmult[2][0]]['alpha'],atol = at, rtol = r_rtol)
        if convergence:
            write('')
            write('Convergence achieved for rmult value :  ',rmult[1][0])
            write('')
            out = {'rmult':rmult[1][0],'r_convergence':True,'results':alphas[rmult[1][0]]}
        else:
            write('')
            write('Convergence for rmult value ',rmult[1][0],' failed')
            write('Return the value of alpha associated to rmult ',rmult[2][0],'. Perform further check!!!')
            write('')
            out = {'rmult':rmult[2][0],'r_convergence':False,'results':alphas[rmult[2][0]]}
        
    return out

In [18]:
build_rmult_list(gs)

[[7.0, 9.0], [8.0, 9.0], [9.0, 9.0]]

So the complete convergence procedure to determine the statical polarizability of a molecule can be performed as follows

In [19]:
posinp_file = molecule+'_posinp.xyz'
gs = lf.Logfile('log-lda.yaml')

r_rt = 1e-2 # relative tol for rmult convergence
f_rt = 1e-2 # relative tol for fieldInt convergence

inp = I.Inputfile()
inp.set_hgrid(gs.log['dft']['hgrids'])
inp.set_xc('LDA')
inp.set_wavefunction_convergence(gnrm=1.0e-5)

polTensor_data[molecule] = perform_rmult_convergence(gs_log=gs,r_rtol=r_rt,f_rtol=f_rt,input=inp,runner=code,posinp=posinp_file,ppf=eval_alpha)
polTensor_data

r_rtol 0.01
f_rtol 0.01

Convergence achieved for field intensity  0.01


Convergence achieved for field intensity  0.01


Convergence achieved for rmult value :  7.0



{'CO': {'r_convergence': True,
  'results': {'F': 0.01,
   'alpha': matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
           [-1.279955e-05,  1.323614e+01, -3.389697e-05],
           [-1.125000e-04, -1.125000e-04,  1.856809e+01]]),
   'f_convergence': True},
  'rmult': 7.0}}